In [2]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import numpy as np
import cv2
import tkinter as tk
from tkinter import filedialog

detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

file_path = ''
while file_path == '':
    file_path = filedialog.askopenfilenames(title='请选择一个视频', initialdir='resources/looking/')
choose_file = file_path[0]
print(choose_file[-6:])

cap = cv2.VideoCapture(choose_file)

fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('../results/looking/look_sideway_res_'+choose_file[-6:], fourcc, fps, size)

while True:
    ret, frame = cap.read()
    if not ret:
        break

        # 转换当前帧为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 自适应直方图均衡化
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_gray = clahe.apply(gray)

    # 对CLAHE结果应用高斯模糊，以减少噪声
    blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整
    gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

    # 使用Canny算法进行边缘检测
    low_threshold = 50  # Canny边缘检测的低阈值
    high_threshold = 150  # Canny边缘检测的高阈值
    canny_edges = cv2.Canny(gaussian_blur, low_threshold, high_threshold)

    # 进行形态学变换 - 闭操作
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(canny_edges, cv2.MORPH_CLOSE, kernel)

    # 将闭操作后的边缘与原CLAHE图像进行叠加
    # 转换闭操作后的边缘图像的数据类型，使其可以与原图像进行加权叠加
    closing_for_combination = cv2.convertScaleAbs(closing)
    enhanced_edges = cv2.addWeighted(clahe_gray, 0.7, closing_for_combination, 0.3, 0)
    
    faces = detect(enhanced_edges, 0)
    if len(faces) == 0:
        cv2.putText(frame, "looking around", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
    else:
        for face in faces:
            face_68_point = predict(gray, face)
            face_68_point = face_utils.shape_to_np(face_68_point)
            
            T = 0.7
            dx_r = abs(face_68_point[39][0] - face_68_point[27][0])
            dx_l = abs(face_68_point[42][0] - face_68_point[27][0])
            if dx_r / dx_l < T:
                cv2.putText(frame, "looking around", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "not looking around", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
    
    out.write(frame)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

03.avi


: 